In [1]:
!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!python -V

Python 3.11.9


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
year = 2023
month = 3

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
df = read_data(input_file)

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
y_pred.std()

6.247488852238703

In [21]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [22]:
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'


In [23]:
!mkdir output

In [24]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [25]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [29]:
!dir .\output

 Volume in drive C is Windows 
 Volume Serial Number is 30A0-01DB

 Directory of c:\Users\arun_prasath\workspace\courses\mlops-zoomcamp\cohorts\2024\04-deployment\homework\output

15-07-2024  11:23    <DIR>          .
15-07-2024  11:22    <DIR>          ..
15-07-2024  11:23        68,641,880 yellow_tripdata_2023-03.parquet
               1 File(s)     68,641,880 bytes
               2 Dir(s)  389,994,663,936 bytes free


In [30]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1623 bytes to starter.py


In [31]:
!pip install pipenv

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
    --------------------------------------- 0.1/3.2 MB 825.8 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/3.2 MB 1.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/3.2 MB 1.8 MB/s eta 0:00:02
   ------ --------------------------------- 0.5/3.2 MB 2.6 MB/s eta 0:00:02
   -------- ------------------------------- 0.7/3.2 MB 2.7 MB/s eta 0:00:01
   ------------ --------------------------- 1.0/3.2 MB 3.2 MB/s eta 0:00:01
   -------------- ------------------------- 1.1/3.2 MB 3.4 MB/s eta 0:00:01
   ------------------ --------------------- 1.5/3.2 MB 3.5 MB/s eta 0:00:01
   --------------------- ------------------ 1.7/3.2 MB 3.8 MB/s eta 0:00:01
   ---------------------- ----------------- 1.8/3.2 MB 3.5 MB/s eta 0:00:01
   ------------------------ --------------- 1.9/3.2 MB 3.4 MB/s eta 0:00:01
   ---------------------